# RAG 🦜️🔗

Veamos cómo agregar un paso de recuperación a un prompt y LLM, lo que suma a una cadena de "generación aumentada con recuperación" (retrieval-augmented generation).

In [10]:
# %pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

In [11]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv
dotenv.load_dotenv()

True

In [12]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [13]:
#Parte 01

vectorstore = FAISS.from_texts(
    ["harrison trabajó en kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Responda la pregunta basándose únicamente en el siguiente contexto:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

Este código está creando una base de datos de vectores utilizando la biblioteca FAISS y OpenAI Embeddings, y luego utilizando esa base de datos para crear un sistema de pregunta-respuesta. Aquí hay un desglose del código:

`vectorstore = FAISS.from_texts(["harrison worked at kensho"], embedding=OpenAIEmbeddings())`

Esta línea crea una base de datos de vectores utilizando la biblioteca FAISS e incrusta el texto "harrison worked at kensho" utilizando el modelo de OpenAI Embeddings. La base de datos de vectores resultante se almacena en la variable `vectorstore`.

`retriever = vectorstore.as_retriever()`

Esta línea crea un objeto recuperador que se puede utilizar para buscar en la base de datos de vectores y recuperar vectores relevantes según una consulta dada.

`template = """Answer the question based only on the following context: {context} Question: {question} """`

Esta línea define una cadena de plantilla que se utilizará para generar prompts para el sistema de pregunta-respuesta. El marcador de posición {context} se reemplazará con el contexto real (es decir, el texto incrustado) y el marcador de posición {question} se reemplazará con la pregunta real.

`prompt = ChatPromptTemplate.from_template(template)`

Esta línea crea un objeto `ChatPromptTemplate` utilizando la cadena de plantilla definida anteriormente.

`model = ChatOpenAI()`

Esta línea crea un modelo de pregunta-respuesta utilizando la clase `ChatOpenAI`. Este modelo se puede utilizar para generar respuestas a preguntas basadas en el contexto proporcionado.

Para modificar este código, puedes cambiar el texto incrustado en la base de datos de vectores modificando la lista de cadenas que se pasa al método `FAISS.from_texts()`. También puedes modificar la plantilla de prompt cambiando la cadena asignada a la variable de plantilla. Finalmente, puedes utilizar un modelo de pregunta-respuesta diferente reemplazando la llamada `ChatOpenAI()` con una instancia de una clase de modelo diferente.

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [15]:
chain.invoke("¿Dónde trabajaba Harrison?")

'Harrison trabajaba en Kensho.'

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [17]:
chain.invoke({"question": "where did harrison work", "language": "español"})

'Harrison trabajó en Kensho.'

# Cadena de recuperación conversacional

Podemos añadir fácilmente historial de conversación. Esto significa principalmente agregar `chat_message_history`.

In [18]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

In [19]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [21]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [22]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [23]:
conversational_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
)

AIMessage(content='Harrison was employed at Kensho.')

In [24]:
conversational_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [
            HumanMessage(content="Who wrote this notebook?"),
            AIMessage(content="Harrison"),
        ],
    }
)

AIMessage(content='Harrison worked at Kensho.')

# Con memoria y devolución de documentos fuente 
Esto muestra cómo usar la memoria con lo anterior. Para la memoria, necesitamos gestionarla fuera, en la memoria. Para devolver los documentos recuperados, solo necesitamos pasarlos todo el camino.

In [25]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory

In [26]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [27]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [28]:
inputs = {"question": "where did harrison work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison was employed at Kensho.'),
 'docs': [Document(page_content='harrison trabajó en kensho')]}

In [29]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})

In [30]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did harrison work?'),
  AIMessage(content='Harrison was employed at Kensho.')]}

In [31]:
inputs = {"question": "but where did he really work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison actually worked at Kensho.'),
 'docs': [Document(page_content='harrison trabajó en kensho')]}